In [7]:
import pandas as pd
import numpy as np
import re
import math

from GoogleNews import GoogleNews
from newspaper import Article

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import gensim
from nltk.tokenize import word_tokenize
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from pprint import pprint
from gensim.models import Phrases


import warnings
warnings.filterwarnings("ignore")

# Google News

In [31]:
keywords_all = pd.read_csv('Keywords - Manual Group.csv')
keywords_all.head()

,Keywords,Other Names,Category
0,accounting fraud,NaN,violation
1,American Depositary Receipts,ADRs,finance
2,anti-money laundering,AML,Rule
3,Asset Management Unit,AMU,Government
4,auditor independence rules,NaN,Rule


In [ ]:
# word = '15c3-5'

In [125]:
keywords = keywords_all.loc[keywords_all['Category'].isin(['Rule', 'Government', 'violation']), ['Category', 'Keywords']]
keywords.head()

,Category,Keywords
0,violation,accounting fraud
2,Rule,anti-money laundering
3,Government,Asset Management Unit
4,Rule,auditor independence rules
6,violation,bribes


In [130]:
googlenews = GoogleNews(start='01/01/2020',end='10/21/2020')

for keyword in keywords['Keywords']:
    googlenews.search(keyword)
    # Return page 3,4,5,6 as well (each page has 10 news)
    #for i in range(4):
     #   googlenews.getpage(i+2) 
    
result = googlenews.result()  
    
results_df = pd.DataFrame(result)
# Remove duplicated news
results_df = results_df.loc[~results_df.duplicated()]

In [34]:
# Download titles and summaries for all news

all_news = []

for i, row in results_df.iterrows():
 #   kw_id = math.floor(i/10)

    article = Article(row['link'])
    
    try:
        article.download()
        article.parse()
        article.nlp()

        if article.title != 'StackPath':
            news_list = [row['date'], row['media'], 
                         article.title, article.summary]
            all_news.append(news_list)

    # if an article cannot be downloaded without authorization, then skip it
    except: continue 

columns = ['date', 'media', 'title', 'summary']
df = pd.DataFrame(all_news, columns=columns)
df.head()

,category,keyword,date,media,title,summary
0,violation,accounting fraud,"Feb 3, 2020",CNN,"Bernard Ebbers, former WorldCom CEO who went t...","New York (CNN Business) Bernard Ebbers, the fo..."
1,violation,accounting fraud,"Feb 4, 2020",,WorldCom chief dies after accounting fraud jai...,"The former CEO of WorldCom, who was sentenced ..."
2,violation,accounting fraud,"In-Depth · Feb 3, 2020",Washington Post,"Bernard Ebbers, WorldCom CEO convicted in hist...","Mr. Ebbers, who was convicted in 2005, had bee..."
3,violation,accounting fraud,"Apr 6, 2020",Motley Fool,Investing Lessons From the Luckin Coffee Accou...,Be careful with Chinese stocks and other emerg...
4,violation,accounting fraud,"Apr 7, 2020",Investment U,Can Luckin Coffee Stock Recover From Accountin...,Investment Opportunities Can Luckin Coffee Sto...


In [38]:
df.to_csv('google_news_1021.csv', index=False)

# Finance Articles

In [1]:
topics = ['compliance', 'cybersecurity', 'surveillance', 'market structure', 
'Regulation', 'short sales', 'funds', 'Securities', 'investment']

In [ ]:
googlenews = GoogleNews(start='01/01/2020',end='10/15/2020')

# Everytime we search for a new keyword, 
# it will save the first two pages result in googlenews.result()
for keyword in topics:
    googlenews.search(keyword)
    
    # Require it to return page 3,4,5,6 as well (each page has 10 news)
    for i in range(4):
        googlenews.getpage(i+1) 

In [152]:
topics_list = []

for i in topics:
    topics_list.extend([i]*50)
    
results_df = pd.DataFrame(result)
results_df['keyword'] = topics_list

# Remove duplicated news
results_df = results_df.loc[~results_df.duplicated()]
results_df.head()

,title,media,date,desc,link,img,keyword
0,USOPC Announces Hiring Of Holly Shick As First...,TeamUSA.org,"Jul 21, 2020",The new chief ethics and compliance officer po...,https://www.teamusa.org/News/2020/July/21/USOP...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",compliance
1,OPEC+ data reveals extent of oil quota non-com...,,"Aug 20, 2020","Full compliance with the deal, including the c...",https://www.spglobal.com/platts/en/market-insi...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",compliance
2,"Nevada OSHA issues 11 workplace citations, 9 f...",FOX5 Las Vegas,"Aug 17, 2020",CITY COMPLIANCE. Cities with compliance rates ...,https://www.fox5vegas.com/coronavirus/nevada-o...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",compliance
3,Hong Kong financial firms step up compliance h...,Reuters,1 month ago,HONG KONG (Reuters) - Financial firms in Hong ...,https://www.reuters.com/article/hongkong-secur...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",compliance
4,Wells Fargo's chief compliance officer quits: FT,Reuters,"Aug 13, 2020",N compliance chief is leaving after two years ...,https://www.reuters.com/article/us-wells-fargo...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",compliance


In [149]:
# Download titles and summaries for all news

all_news = []

for i, row in results_df.iterrows():

    article = Article(row['link'])
    
    try:
        article.download()
        article.parse()
        article.nlp()

        if article.title != 'StackPath':
            news_list = [row['keyword'], row['date'], row['media'], 
                         article.title, article.summary]
            all_news.append(news_list)

    # if an article cannot be downloaded without authorization, then skip it
    except: continue 

columns = ['keyword', 'date', 'media', 'title', 'summary']
df = pd.DataFrame(all_news, columns=columns)
df.head()

,keyword,date,media,title,summary
0,compliance,"Jul 21, 2020",TeamUSA.org,USOPC Announces Hiring Of Holly Shick As First...,"COLORADO SPRINGS, Colo. – The United States Ol..."
1,compliance,"Aug 20, 2020",,OPEC+ data reveals extent of oil quota non-com...,The OPEC+ alliance in May implemented the larg...
2,compliance,"Aug 17, 2020",FOX5 Las Vegas,"Nevada OSHA issues 11 workplace citations, 9 f...",Close Get email notifications on {{subject}} d...
3,compliance,1 month ago,Reuters,Hong Kong financial firms step up compliance h...,HONG KONG (Reuters) - Financial firms in Hong ...
4,compliance,"Aug 13, 2020",Reuters,Wells Fargo's chief compliance officer quits: FT,(Reuters) - Wells Fargo & Co's WFC.N complianc...


In [139]:
df['keyword'].value_counts()

surveillance        59
market structure    59
compliance          58
short sales         58
funds               57
cybersecurity       57
Regulation          55
Securities          25
Name: keyword, dtype: int64

# Keyword 15c3-5

In [15]:
keyword_col = []
for i in ['15c3-5']:
    keyword_col.extend([i]*10*10)
    
len(keyword_col)

100

In [25]:
def google_news(keyword_list: list, num_pages: int=2):
    
    googlenews = GoogleNews()

    # Everytime we search for a new keyword, 
    # it will save the first two pages result in googlenews.result()
    
    pre_n = 0
    keyword_col = []
    
    for keyword in keyword_list:
        googlenews.search(keyword)

        # Require it to return other pages and save results to googlenews.result()
        for i in range(1, num_pages):
            googlenews.getpage(i) 
        
        cur_n = len(googlenews.result())
        keyword_col.extend([i]*(cur_n-pre_n))
        num_results = cur_n
            
    result = googlenews.result()  
    results_df = pd.DataFrame(result)
          
    results_df['keyword'] = keyword_col
    
    # Remove duplicated news
    results_df = results_df.loc[~results_df.duplicated()]
    
    all_news = []

    for i, row in results_df.iterrows():

        article = Article(row['link'])

        try:
            article.download()
            article.parse()
            article.nlp()

            if article.title != 'StackPath':
                news_list = [row['keyword'], row['date'], row['media'], 
                             article.title, article.summary]
                all_news.append(news_list)

        # if an article cannot be downloaded without authorization, then skip it
        except: continue 

    columns = ['keyword', 'date', 'media', 'title', 'summary']
    df = pd.DataFrame(all_news, columns=columns)
    
    return df

In [26]:
news_df = google_news(['15c3-5'], num_pages=10)

In [28]:
news_df.to_csv('Google_news_15c3-5.csv', index=False)